In [1]:
!pip install apache_beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 16.0 MB/s eta 0:00:00
  Created wheel for crcmod: filename=crcmod

In [2]:
import apache_beam as beam

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/Cloud-AI-Analytics/Apache\ Beam\ -Python/data

/content/drive/MyDrive/Colab Notebooks/Cloud-AI-Analytics/Apache Beam -Python/data


In [5]:
!ls

data	       grocery.txt			  result	    students_exclude.txt
dept_data.txt  regular_filter.txt-00000-of-00001  Students_age.txt  students.txt


In [7]:
!{('head -n 10 students_exclude.txt')}

1
3
7
9

## **Side Inputs**:

•	A side input is an additional input that your DoFn can access each time it processes an element in the input PCollection.

•	In addition to the main input PCollection, you can provide additional inputs to a ParDo transform in the form of side inputs.


In [8]:
p1 = beam.Pipeline()

input_list = list()
with open ('students_exclude.txt','r') as exclude_file:
  for stud_id in exclude_file:
    input_list.append(stud_id.rstrip())

print(input_list)

['1', '3', '7', '9']


In [9]:
class SplitRow(beam.DoFn):
  def process(self,element,input_list):
    customer = element.split(',')
    if customer[0] not in input_list:
      return [customer]

customers = (
    p1
    |beam.io.ReadFromText('Students_age.txt')
    |beam.ParDo(SplitRow(),input_list)  #can pass any number of side inputs in this ParDo function
    |beam.io.WriteToText('data/output')
)
p1.run()

In [10]:
!{('head -n 10 data/output-00000-of-00001')}

['2', 'farooqui', 'hyd', '26']
['4', 'neethu', 'mla', '27', '']
['5', 'joey', 'ny', '57']
['6', 'ross', 'la', '60']
['8', 'lois', 'us', '50']
['10', 'sai', 'chn', '29']


## **Side Outputs/Additional Outputs:**



*   Additional outputs in parDo transformation

*   While ParDo always produces a main output PCollection (as the return value from apply), you can also have your ParDo produce any number of additional output PCollections.



In [15]:
p1 = beam.Pipeline()

side_list = list()
with open ('students_exclude.txt','r') as exclude_file:
  for cust_id in exclude_file:
    side_list.append(cust_id.rstrip())

print(side_list)

class SplitRow(beam.DoFn):
  def process(self,element,side_list):
    customer = element.split(',')
    if customer[0] not in side_list:
      return [customer]

class ProcessCustomers(beam.DoFn):
  def process(self,element,country,start_char):
    if(element[2]==country):
      yield  element
    else:
      yield  beam.pvalue.TaggedOutput('Other_student',element)
    if(element[1].startswith('r')):
       yield  beam.pvalue.TaggedOutput('Names_r',element)



customers = (
    p1
    |beam.io.ReadFromText('Students_age.txt')
    |beam.ParDo(SplitRow(),side_list)
    |beam.ParDo(ProcessCustomers(),'chn','r').with_outputs('Names_r','Other_student',main='Chennai_Cust')
)

chennai_customers = customers.Chennai_Cust
other_cities_customers = customers.Other_student
customer_withname_r = customers.Names_r

chennai_customers | 'Write Chennai Students PCollection' >> beam.io.WriteToText("data/chennai")
other_cities_customers  | 'Write Students PCollection that lives in other cities' >> beam.io.WriteToText("data/students_other_cities")
customer_withname_r  | 'Write Students names with r PCollection' >> beam.io.WriteToText("data/customers_names_r")


p1.run()


['1', '3', '7', '9']


In [16]:
! cat data/chennai-00000-of-00001

['10', 'sai', 'chn', '29']


In [17]:
!cat data/students_other_cities-00000-of-00001

['2', 'farooqui', 'hyd', '26']
['4', 'neethu', 'mla', '27', '']
['5', 'joey', 'ny', '57']
['6', 'ross', 'la', '60']
['8', 'lois', 'us', '50']


In [18]:
!cat data/customers_names_r-00000-of-00001

['6', 'ross', 'la', '60']
